In [17]:
from flask import Flask, render_template, request, jsonify
import os
import cv2
import numpy as np
from tensorflow.lite.python.interpreter import Interpreter

app = Flask(__name__)

def tflite_detect_objects(model_path, image, labels_path, min_conf=0.5):
    # Load the label map into memory
    with open(labels_path, 'r') as f:
        labels = [line.strip() for line in f.readlines()]

    # Load the TensorFlow Lite model into memory
    interpreter = Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get model details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    # Load image and resize to expected shape [1xHxWx3]
    imH, imW, _ = image.shape
    image_resized = cv2.resize(image, (width, height))
    input_data = np.expand_dims(image_resized, axis=0)

    # Normalize pixel values
    input_mean = 127.5
    input_std = 127.5
    input_data = (np.float32(input_data) - input_mean) / input_std

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run inference
    interpreter.invoke()

    # Retrieve detection results
    boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates of detected objects
    classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
    scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

    detections = []

    # Loop over all detections and store if confidence is above minimum threshold
    for i in range(len(scores)):
        if ((scores[i] > min_conf) and (scores[i] <= 1.0)):
            ymin = int(max(1,(boxes[i][0] * imH)))
            xmin = int(max(1,(boxes[i][1] * imW)))
            ymax = int(min(imH,(boxes[i][2] * imH)))
            xmax = int(min(imW,(boxes[i][3] * imW)))
            object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
            confidence = scores[i]
            detections.append({'label': object_name, 'confidence': confidence, 'bbox': [xmin, ymin, xmax, ymax]})

    return detections

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Mendapatkan data dari form
        model_path = 'model/model-fix-final.tflite'
        labels_path = 'labelmap.txt'
        min_conf = 0.5

        # Mendapatkan gambar yang diunggah
        image_file = request.files['image']
        
        # Membaca gambar yang diunggah
        image = cv2.imdecode(np.frombuffer(image_file.read(), np.uint8), cv2.IMREAD_COLOR)

        # Memanggil fungsi deteksi objek
        results = tflite_detect_objects(model_path, image, labels_path, min_conf)

        # Mengembalikan hasil deteksi dalam format JSON
        return jsonify(results)

    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)

In [21]:
tflite_detect_objects(model_path='model/model-fix-final.tflite', image=cv2.imread('cah-kangkung-telur-puyuh.jpg'), labels_path='labelmap.txt', min_conf=0.5)

[{'label': 'kangkung', 'confidence': 0.96620977, 'bbox': [139, 101, 614, 385]}]